In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.vision import *
from fastai import * 
from fastai.text import *
from pathlib import Path 
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

In [4]:
pip install torchtext

Note: you may need to restart the kernel to use updated packages.


In [110]:
import torchtext
from torchtext.data import Iterator, BucketIterator
from torchvision import transforms, utils


In [6]:
path = Config.data_path()
path.mkdir(parents=True, exist_ok=True)

path.ls()

In [127]:
df_train = pd.read_csv(path/'train.csv')
df_test = pd.read_csv(path/'test.csv')
# df_train
print(df_train.shape)
print(len(df_train))

(1306122, 3)
1306122


In [8]:
import string
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]



mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

def mispelled_words(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = mispelled_words(mispell_dict)

def replace_typical_mispell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace,text)

def clean_text(x):

    x = str(x)
    for punct in puncts:
        x.translate(str.maketrans('', '', string.punctuation))
    return x



In [122]:
len(df_train)


1044898

In [9]:
df_train["question_text"] = df_train["question_text"].str.lower()
df_test["question_text"] = df_test["question_text"].str.lower()

df_train['question_text'] = df_train['question_text'].progress_apply(lambda x: clean_text(x))
df_test['question_text'] = df_test['question_text'].progress_apply(lambda x: clean_text(x))

df_train['question_text'] = df_train['question_text'].progress_apply(lambda x: clean_numbers(x))
df_test['question_text'] = df_test['question_text'].progress_apply(lambda x:clean_numbers(x))

df_train['question_text'] = df_train['question_text'].progress_apply(lambda x: replace_typical_mispell(x))
df_test['question_text'] = df_test['question_text'].progress_apply(lambda x: replace_typical_mispell(x))

train_x = df_train['question_text'].fillna('_##_').values
test_x = df_test['question_text'].fillna('_##_').values

my bar!:  58%|█████▊    | 752172/1306122 [06:22<10:41, 863.31it/s] 

KeyboardInterrupt: 

import zipfile
zf = zipfile.ZipFile(path/'test.csv.zip')
df_test = pd.read_csv(zf.open('test.csv'))
zf = zipfile.ZipFile(path/'train.csv.zip')
df_train = pd.read_csv(zf.open('train.csv'))



In [139]:
train_length = len(df_train)
print(train_length)
train_length*0.80 

1306122


1044897.6000000001

In [140]:
df_train, val = torch.utils.data.random_split(df_train,[round(train_length * 0.80),round(train_length * 0.20)])


In [ ]:
train_y = df_train['target'].values

In [10]:
def prepare_sequences():
    tokenizer = lambda text: text.split() # the function above is the function we will be using to tokenize the text
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer, lower=True)
    LABEL = torchtext.data.Field(sequential=False, use_vocab=False) # sequential and use_vocab=False since no text (binary)
    train_datafields = [("qid", None), ("question_text", TEXT), ("target", LABEL)]
    train = torchtext.data.TabularDataset( # If we had a validation set as well, we would add an additional .splits(...)
                        # path="data/train_cleaned_v2.csv", # the root directory where the data lies
                        path = "/Users/caitlinaspinall/.fastai/data/train.csv",
                        format='csv',
                        # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                        skip_header=True, 
                        fields=train_datafields)
    test_datafields = [("qid", None),
                     ("question_text", TEXT), ("target", LABEL)] 
    test = torchtext.data.TabularDataset( 
                path="/Users/caitlinaspinall/.fastai/data/test.csv",
                format="csv",
                skip_header=True,
                fields=test_datafields)
    return TEXT, LABEL, train, test
# Chat Conversation End
# Type a message...



In [11]:
TEXT,LABEL,train,test = prepare_sequences()


my bar!:  58%|█████▊    | 752251/1306122 [06:40<10:41, 863.31it/s]

In [20]:
TEXT.build_vocab(train, test, vectors = "glove.6B.100d")

In [147]:

from torchtext.data import Iterator, BucketIterator
train_iter = BucketIterator(
    train, # we pass in the datasets we want the iterator to draw data from
    batch_size=64, 
    sort_key= lambda x: len(x.question_text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False, # sorting would add bias
    repeat=False)
test_iter = Iterator(test, batch_size = 64, device = None, sort = False,  sort_within_batch = False, repeat = False) #Changed device from -1 to None
# val_iter = Iterator(val, batch_size = 64, device = None, sort = False,  sort_within_batch = False, repeat = False)

AttributeError: 'Subset' object has no attribute 'sort_key'

In [23]:
iter(train_iter).__next__().question_text
iter(train_iter).__next__().question_text.shape



torch.Size([38, 64])

In [34]:
TEXT.vocab.vectors.shape

torch.Size([527626, 100])

In [88]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        self.embedding = nn.Embedding(len(TEXT.vocab),100)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        
    def forward(self,train):
        print(train.shape)
        return train
batch = next(iter(train_iter)).question_text
net = NeuralNet()
net(batch)

torch.Size([47, 64])


tensor([[    3,   389,     3,  ...,    10,    19, 16575],
        [   49,    18,     4,  ...,    50,  2584,     4],
        [61523,     5,     2,  ..., 56110,   269,     5],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [40]:
num_epochs = 5

In [ ]:
def train_model(train_data, val_data, model,num_epochs, bs, optimizer, loss_function, show = False):
    errors = []
    eval_errors = []
    f1_train = []
    auc_eval = []
    f1_eval =[]
    auc_eval=[]
    num_training_batches = len(train_data)
    
    for epoch in range(num_epochs):
        model.train()
        train_iter = iter(train_data)
        model.zero_grad()
        train_loss = 0
        all_preds = []
        all_actual = []
        
        for batch_num in range(num_training_batches):
            batch = next(train_iter)
            optimizer.zero_grad()
            preds = model(batch)
            loss = loss_function(preds)
            hard_preds = torch.round(preds)
            all_preds.extend(hard_preds.squeeze(-1).tolist())
            #
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
        train_avg_loss = train_loss/(num_training_batchsize*batch_size)
        train_confusion_matriz = confusion_matric(all_actaul, all_preds).ravel()
        
        f1_train.append(f1_score(all_actual,all_preds))
        errors.append(train_loss_avg)
        model.eval()
        num_val_batches = len(val)
        val_iter = iter(val)
        eval_loss =0
        all_pred = []
        all_actual = []
        
        return
train_model()  

In [101]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.001, momentum = 0.9)

In [61]:
df_train.shape
print()

1306122


In [55]:
1306122 * 0.2

261224.40000000002

In [112]:
1306122-261224

1044898